# Pytorch Distributed Data parallel(DDP) processing tutorial

1. Import pytorch module for do DDP processing

In [ ]:
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader

from torchvision import transforms
from torchvision.datasets import ImageFolder

2. Define functions

In [ ]:
def main(args):
    n_gpus = torch.cuda.device_count()
    args.world_size = n_gpus

    if args.use_ddp: # a flag for using Pytorch DDP
        args.world_size = n_gpus * args.world_size
        args.num_workers = n_gpus * 4 # it's common to multiply 4 with how many gpu you have
        args.batch_size = n_gpus * args.batch_size # to split batch per each gpu
        args.val_batch_size = n_gpus * args.val_batch_size # same above
        mp.spawn(main_worker, nprocs=n_gpus, args=(n_gpus, args)) # pytorch multiprocessing spawn
    else: # a flag for not using Pytorch DDP
        args.gpu = 0 # set first gpu id
        main_worker(args.gpu, n_gpus, args)

def main_worker(gpu, ngpus_per_node, args):
    args.gpu = gpu
    torch.cuda.set_device(args.gpu)

    if args.gpu is not None:
        print("Use GPU: {} for training".format(args.gpu))

    if args.distributed:
        if args.is_distributed:
            args.rank = args.rank * ngpus_per_node + gpu
        dist.init_process_group(backend='nccl', # 'gloo', 'mpi', or 'nccl', check out their capabilities
                                init_method='tcp://127.0.0.1:88', #{localhost}:OPENPORT
                                world_size=args.world_size, 
                                rank=args.rank)

    # load dataset
    transform = transforms.Compose([transforms.Resize([256, 256]),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                        std=[0.5, 0.5, 0.5])])
    dataset = ImageFolder("your directory", transform=transform)

    sampler = DistributedSampler(dataset=dataset, shuffle=True) # split train data per each process
    dataloader = DataLoader(dataset, 
                            batch_size=int(args.batch_size / args.world_size),
                            shuffle=False,
                            num_workers=int(args.num_workers / args.world_size)
                            sampler=sampler,
                            pin_memory=True)